In [1]:
class Throttle:
    """
    Add a delay between downloads for each domain
    """
    def __init__(self, delay):
        #amount of delay between downloads for each domain
        self.delay = delay
        #timestamp of when a domain was latest accessed
        self.domains = {}
        
    def wait(self, url):
        domain = urlparse(url).netloc
        last_accessed = self.domains.get(domain)
        
        if self.delay > 0 and last_accessed is not None:
            sleep_secs = self.delay - (time.time() - last_accessed)
            if sleep_secs > 0:
                #domain has been accessed recently
                #so need to sleep
                time.sleep(sleep_secs)
                
        #update the last accessed time
        self.domains[domain] = time.time()

In [2]:
from random import choice
import requests

In [3]:
class Downloader:
    def __init__(self,delay =5, user_agent = 'naruto', proxies = None, cache = {}):
        self.throttle = Throttle(delay)
        self.user_agent = user_agent
        self.proxies = proxies
        self.num_retries = None # we will set this per request
        self.cache = cache
    
    def __call__(self,url,num_retries = 2):
        self.num_retries = num_retries
        try:
            result = self.cache[url]
            print('Load from cache:', url)
        except KeyError:
            result = None
        if result and self.num_retries and 500<= result['code'] < 600:
            result = None
        if result is None:
            self.throttle.wait(url)
            proxies = choice(self.proxies) if self.proxies else None
            headers = {'User-Agent': self.user_agent}
            result = self.download(url, headers, proxies)
            if self.cache:
                #save result to cache
                self.cache[url] = result
        return result['html']
    
    def download(self,url,headers,proxies,num_retries):
        ...
        return {'html':html,'code':resp.status_code}
    

In [6]:
import os


In [10]:
os.chdir("/Users/rivers/Desktop/all_project/crawlerbook/wswp/code")

In [57]:
import os
import json
import re
import zlib

from datetime import datetime, timedelta
from urllib.parse import urlsplit


class DiskCache:
    """ DiskCache helps store urls and their responses to disk
        Intialization components:
            cache_dir (str): abs file path or relative file path
                for cache directory (default: ../data/cache)
            max_len (int): maximum filename length (default: 255)
            compress (bool): use zlib compression (default: True)
            encoding (str): character encoding for compression (default: utf-8)
            expires (datetime.timedelta): timedelta when content will expire
                (default: 30 days ago)
    """
    def __init__(self, cache_dir='../data/cache', max_len=255, compress=True,
                 encoding='utf-8', expires=timedelta(days=30)):
        self.cache_dir = cache_dir
        self.max_len = max_len
        self.compress = compress
        self.encoding = encoding
        self.expires = expires

    def url_to_path(self, url):
        """ Return file system path string for given URL """
        components = urlsplit(url)
        # append index.html to empty paths
        path = components.path
        if not path:
            path = '/index.html'
        elif path.endswith('/'):
            path += 'index.html'
        filename = components.netloc + path + components.query
        # replace invalid characters
        filename = re.sub(r'[^/0-9a-zA-Z\-.,;_ ]', '_', filename)
        # restrict maximum number of characters
        filename = '/'.join(seg[:self.max_len] for seg in filename.split('/'))
        return os.path.join(self.cache_dir, filename)

    def __getitem__(self, url):
        """Load data from disk for given URL"""
        path = self.url_to_path(url)
        if os.path.exists(path):
            mode = ('rb' if self.compress else 'r')
            with open(path, mode) as fp:
                if self.compress:
                    data = zlib.decompress(fp.read()).decode(self.encoding)
                    data = json.loads(data)
                else:
                    data = json.load(fp)
            return data
        else:
            # URL has not yet been cached
            raise KeyError(url + ' does not exist')

    def __setitem__(self, url, result):
        """Save data to disk for given url"""
        path = self.url_to_path(url)
        folder = os.path.dirname(path)
        if not os.path.exists(folder):
            os.makedirs(folder)
        mode = ('wb' if self.compress else 'w')
        # Note: the timespec command requires Py3.6+ (if using 3.X you can
        # export using isoformat() and import with '%Y-%m-%dT%H:%M:%S.%f'

        with open(path, mode) as fp:
            if self.compress:
                data = bytes(json.dumps(result), self.encoding)
                fp.write(zlib.compress(data))
            else:
                json.dump(result, fp)


In [62]:
%time link_crawler('http://example.webscraping.com/', '/(index|view)', cache=DiskCache())

error: Error -5 while decompressing data: incomplete or truncated stream

In [63]:
import redis


ImportError: No module named 'redis'

In [1]:
import redis


ImportError: No module named 'redis'